In [ ]:
#| default_exp power


In [ ]:
#| hide
from nbdev.showdoc import *

# Direct Power Analysis

Good grief this is complicated

This section includes the formula for

- fullscale wake fraction in trial conditions J-17
- load factor in trial conditions J-18
- Total resistance in trial conditions J-19
- Total resistance in ideal conditions J-22
- scale corellation factor J-21
- load factor idea J-23
- propeller shaft speed -J-29


Some things should come with a bit more clarification e.g. 'wake-fraction', what is it?

In [ ]:
#| export
import numpy as np
from fastcore.test import *

## Correction delivered power

Calculates the power when correcting for the resistances experienced by the vessel as well as the deciation of propeller efficiency relative to ideal

$$\Delta P = \frac{\Delta R V_s}{\eta_{Dms}}+ P_{Dms}(1-\frac{\eta_{Dms}}{\eta_{Did}}), $$ where $\Delta R$ is the total resitance increase experienced by the ship, $V_s$ is the speed through water, $\eta_{Did}$ is the propulsive efficiency coefficient in the ideal conditions, $P_{Dms}$ is the shaft power during the test,$\eta_{Dms}$ is the propulsive efficiency coefficient in the trial conditions.

The value for $\Delta R$ is calculated as the some of the wind, wave, and water resistances. The functions for these values are found at `wind_resistance`, `stawave1_fn`, and `temp_salinity_water_resistance`

**ITTC equations**: J-2

In [ ]:
#| export
def correction_delivered_power(
    p_dms:float, #delivered power [W]
    resistance_increase:float, #Resistance increase derived from data measured in seatrial
    stw:float, #speed through water [m/s]
    eta_id:float, #propulsive efficiency in the ideal conditions
    eta_ms:float, # propulsive efficiency in the seattrial

)-> float:
    
    "calculates the corrected delivered power, used as part of the direct power analysis"
    
    return resistance_increase * stw /eta_id + p_dms * (1- eta_ms/eta_id)
    

Example of using function, generally this is not used directly

In [ ]:
correction_delivered_power(1e4, 1e3, 10, 0.8, 0.7)

13750.000000000002

## Propulsive efficiency correction
This can be used to calculate both the ideal and trial conditions reffered to in equations J-3 and J-5
$$\eta_D = \eta_o \eta_R \frac{1-t}{1-w_s} $$

**ITTC equations** J-3, J-5


In [ ]:
#| export
def propulsive_efficiency_corr(n_o:float, #open water efficiency
                          n_r:float, #relative rotative efficiency
                          t:float, #thrust deduction factor
                         w_s:float, #full-scale wake fraction
                         ):
    "Calculates propeller efficiency adjusting for additional resistances"
    return n_o*n_r*(1-t)/(1-w_s)
    

In [ ]:
propulsive_efficiency_corr(0.58, 0.7, 0.1, 0.5)

0.7308

In [ ]:
#| hide

test_eq(propulsive_efficiency_corr(0.58, 0.7, 1, 0),0)
test_eq(propulsive_efficiency_corr(1, 1, 0, 0.5),2)
#checks to make sure there is a divide by 0 error
test_fail(lambda: propulsive_efficiency_corr(0.58, 0.7, 0.0, 1),  contains="division by zero" )


## Full scale wake fraction

There are two different functions related to the full scale wake fraction, both are presented in this sub-section. 

The two related but distinct values are

- $w_{S}$: Full-scale wake fraction
- $e_i$ Scale correlation factor of the wake faction

### The full scale wake fraction

$$w_S =  1- (1- w_M)e_i,$$ where $W_S$ is the fullscale wake fraction, $W_m$ is the model wake fraction derived from tank tests, and $e_i$ is the scale correlation factor derived from **XXXX derived from what?**.

**ITTC equation**: J-4, J-20


In [ ]:
#| export
def full_scale_wake_fraction(wake_fraction_model:float,
                            scale_correlation_factor:float
                            )-> float:
    
    "used to scale from model results to full-scale vessel"
    
    return 1- (1- wake_fraction_model) * scale_correlation_factor

The full scale wake fracrtion returns a value between 0 and 1

In [ ]:
full_scale_wake_fraction(0.4, 0.8)

0.52

In [ ]:
#| hide

test_eq(full_scale_wake_fraction(1, 0.8), 1)
test_close(full_scale_wake_fraction(0, 0.8), 0.2, eps = 1e-6)

### Full-scale wake fraction from speed

This approach calculates the wake fraction using the measured water speeds

$$w_S =1- \frac{V_A}{V_S},$$ where $V_A$ is the speed of flow into the propeller, and $V_S$ is the ship's speed through water. 

**ITTC equations**: J-17

In [ ]:
#| export

def full_scale_wake_speed(flow_speed:float, #The speed of flow through the propeller
                         stw:float, #Ship's speed through water
                         )-> float:
    
    "Calculate the wake fraction using the measured water speeds"
    
    return 1 - (flow_speed/stw)

This approach to obtianing the full scale wake speed is typically used for getting the trial conditions

In [ ]:
full_scale_wake_speed(10,50)

0.8

In [ ]:
#| hide
test_eq(full_scale_wake_speed(50,50),0)

### Scale correlation factor

The scale correlation factor is simply the re-arranged fullscale wake fraction. And is shown as

$$e_i = \frac{1-w_S}{1-w_M},$$ where values are the same as previously.

**ITTC equation**: J-21

In [ ]:
#| export
def scale_correlation_factor(
    trial:float, #The full-scale wake fraction in the trial
    model:float  #The wake fraction of the model derived from tank tests
)-> float: #The dimensionless coefficient joining the full scale and model fractions
    "Calcualte the scale correlation factor using the model fraction from tank tests, and the full-scale fraction from trials"
    return (1  - trial)/(1-model)

Deriving the scale correlation factor is easy given the inputs

In [ ]:
scale_correlation_factor(0.6,0.8)

2.0000000000000004

In [ ]:
#| hide

test_eq(scale_correlation_factor(1,0.8),0)

## Self propulsion factors

This function allows for ship factors to be adjust accounting for the difference between ideal/model conditions and the conditions during the trial. This function is equivalent to equation J-6 to J-8.
The function is essentially the following equation

$$x_{test} = x_{ideal} + \Delta x_R (\frac{\Delta R}{R_{ideal}}),$$
Where $x$ is the variable to be adjusted $x_{ideal}$ is the value of the variable under ideal conditions, $\Delta x_R$ the coefficient of of change for each unit of $\frac{\Delta R}{R_{ideal}}$, $\Delta R$ the resistance increase derived from data measured during the sea trial and $R_{ideal}$ is the resistance under ideal conditions. The values of $x_{ideal}$ are obtained from a model test. The value of $\Delta x_R$ is found by fitting a linear model using data gathered during a specific tank tests. The process of obtaining the value of $x_R$ is described in detail in section J.2.

In practice however, these sets of values are not needed as the deviations are often negligable in comparison to the variation of $\eta_O$. That using 
$$ x_{test} \approx x_{ideal},$$
is acceptable

**ITTC equations**: J-6, J-7, J-8

In [ ]:
#| export
def self_propulsion_factors(
    x_ideal:float, #The variable in ideal conditions. It is acceptable to use this value without adjustments
    delta_x:float = 0, #The change per unit of the resistance ratios. Default is 0
    delta_r:float = 1, #increase in resistance from ideal conditions
    delta_r_ideal:float = 1 #Resistance in ideal conditions
) -> float:
    
    "Adjusting the self propulsion factors is only possible if the required model tests have been performed. By default this function returns the ideal value"
    
    return x_ideal + delta_x * (delta_r/delta_r_ideal)

Generally the adjustment for the self propulsion factors will be small. But if available should be applied

In [ ]:
self_propulsion_factors(0.8, 0.1, 1000, 10000)

0.81

In [ ]:
#| hide

test_eq(self_propulsion_factors(0.8), 0.8)


## Calculate Thrust coefficient, Torque coefficient, and Load factor

The thrust coefficient, torque coefficient and the load factor provide useful values for calculating the adjusted propeller efficiency. There are several different approaches. All three approaches are described in this sub-section and are used at various points in the process.

### The quadratic method

The quadratic method doesn't actually calculate the coefficients themselves but using them as the dependent variable of a quadratic curve where the dependent variable is the propeller advance coefficient in the ideal condition. The result is that the coefficients of the quadratic curve can be either be solved to find the point closest to 0 where the line crosses the x-axis. Alternatively the quadratic coefficients are used replacing the propeller advance coefficients in the ideal condition with those of the trial condition, this creates a quadratic equation that return the thrust coefficient in the trial condition.
The propeller advance coefficients in the ideal condition are provided by tank tests, usually 10 data points are supplied in order to fit the curve

$$y = aX^2 + bX +c$$

The coefficients of this model ($a,b,c$) are not useful inthemselves but are input parameters to other functions for example `torque_coef`

The equation is also used to calculate the thrust coefficient in the trial condition/


**ITTC equations**: J-9, J-10, J-11, J-14, J-25, J-26



In [ ]:
#| export
def get_curve_coefficient(y:float, #The dependent variable coefficient
                      x:float #The propeller advance coefficient
                     )->float: #returns model coefficients
    
    "Obtain the coefficients used to calculate the Thrus, and Torque coefficients and the load factor coefficients"
    
    #create the X matrix to have a quadratic form
    X = np.concatenate((x**2,x, np.ones(len(x)))).reshape([3,10]).transpose()
    #Get determinate

    square_mat = np.matmul(X.transpose(), X)

    detX = np.linalg.inv(square_mat)

    temp = np.matmul(detX, np.transpose(X))
    #Return the beta value
    b = np.matmul(temp, y)
    
    return b
    

For ship whose model tests give a propeller advance coefficient to thrust coefficent $K$ relationship as shown below, we can (back) calculate the coefficients of the quadratic formula using the get_curve_coefficient function.

In [ ]:
J = np.linspace(1,10, 10)
K = J**2 + 2*J +3 #In reality we obviously do not know the coefficients before hand!
get_curve_coefficient(K, J)

array([1., 2., 3.])

When calculating the loading factor coefficients $\tau$ the values for the propeller advance coefficient needs to be inverted

In [ ]:
J = 1/np.linspace(1,10, 10)
K = J**2 + 2*J +3 #In reality we obviously do not know the coefficients before hand!
get_curve_coefficient(K, J)

array([1., 2., 3.])

In [ ]:
#| hide
#tests are amazing, found out I had an error in the code because the test failed at seemingly random points.
#was actually an error in the function causing it to search for the variable in the global environment
x = np.linspace(1,10, 10)
y = x**2 + 2*x +3
test_close(get_curve_coefficient(y, x)[1], 2, eps = 1e-5)

### Torque coefficient

The torque coefficient is obtained using 

$$K = \frac{P}{2 \pi n^3 D^5} \eta,$$ Where $P$ is power, $n$ is rotations per second, $D$ is shaft diameter and $eta is relative rotative efficiency.

Generally the torque coefficient for the ideal condition is known through tank tests. However, this equation is also used to find the thrust coeficient during the trial. When the trial thrust coefficient is calculated $P$, $n$, and $\eta$ are all from trial data.

**ITTC equations**: J-12

In [ ]:
#| export
def torque_coef(power:float, #The delivered power
                     shaft_speed:float, #measure propeller shaft speed [rev/s]
                     diameter:float, #properller_diameter [m]
                     efficiency:float, #relative rotative efficiency
                     water_density:float = 1026 #water density [kg/m^3]
                     )->float: #dimensionless thrust coefficient
    
    "calcualte the torque coefficient under ideal or trial conditions"
    
    denominator = 2 * np.pi * shaft_speed * diameter
    
    return (power/denominator) *  efficiency
    
    

under trial conditions the thrust coefficient is calculated directly

In [ ]:
torque_coef(1e4, 50, 1.5, 0.8)

16.976527263135505

In [ ]:
#| hide

test_eq(torque_coef(1e4*np.pi, 1, 1, 1, 1000), 5000)

## Trial propeller advance

The propeller advance coefficient for the trial conditions can be calculated using

$$J = \frac{-b - \sqrt{b^2 -4a(c-K)}}{2a},$$ where $a,b,$ and $c$ are the ouput coefficients from `get_curve_coefficient`. and $K$ is the torque coefficient calculated using `torque_coef`. Note that the propeller advance is only the negative part of the quadratic equation as properller advance must always be positive. This equation is used to derive the propeller advance coefficient from empirical measurements.

**ITTC equations**: J-13, J-24


In [ ]:
#| export
def propeller_advance_coefficient(torque_coefficient:float, #The torque coefficient
                                  a:float, #coefficient 'a' from get_curve_coefficient
                                  b:float, #coefficient 'b' from get_curve_coefficient
                                  c:float #coefficient 'c' from get_curve_coefficient

                                  ) -> float:
    
    "Calculate the propeller advance coefficient for"
    square_root = np.sqrt(b**2 - 4*a*(c - torque_coefficient))
    
    return (-b - square_root )/(2*a)

The propeller advance can then be calcualted by combining outputs of the previous functions `get_curve_coefficient` and `torque_coef` as inputs to `propeller_advance_coefficient`.



In [ ]:
#These values are obviously not realistic
J = np.linspace(1,10, 10)
K = 2*J**2 + -5*J +3 #In reality we obviously do not know the coefficients before hand!
curve_coefs = get_curve_coefficient(K, J)
trial_torque_coef = torque_coef(1e4, 50, 1.5, 0.8)
propeller_advance_coefficient(trial_torque_coef, curve_coefs[0], curve_coefs[1], curve_coefs[2]  )

-1.6741688787701197

In [ ]:
#| hide
#(x - 1)(2x - 3) = 2x^2 -5x +3
#put in test when I have reasonable values to test
#check the equation is implemented properly
test_eq(propeller_advance_coefficient(0, 2, -5, 3 ),1)

## Open water efficiency

The efficiency of the properller in open water can be calcualted using 

$$\eta_O = \frac{J}{2\pi}\frac{K_T}{K_Q},$$ where J is the propeller advance coefficient, $K_T$ is the thrust coefficient, and $K_Q$ is the torque coefficient. It can be used for calulating both the trial conditions, but is typically used for trial conditions as the ideal conditions are known from tank tests.

**ITTC equations**: J-15, J-27

In [ ]:
#| export
def open_water_efficiency(propeller_advance_coef:float, #The propeller advance coefficient of the ship
                         thrust_coef:float, # thrust coefficient
                         torque_coef:float 
                         )-> float:
    
    "Calculate the open water propeller efficiency"
    
    return (propeller_advance_coef/(2*np.pi))*(thrust_coef/torque_coef)

The proppeller open water efficiency for a ship with a propeller advance coefficent of 5, thrust coefficent of 0.8 and torque coefficient of 0.9. Is shown below

In [ ]:
open_water_efficiency(5, 0.8, 0.9)

0.707355302630646

In [ ]:
#| hide
test_eq(open_water_efficiency(5, 2*np.pi, 1), 5 )

## Speed of flow into the propeller

The rate at which water flows past the propeller is given by

$$V_A = JnD$$, where $J$ is the propeller advance coefficient, $n$ is the rotations per second, and $D$ is the diameter of the propeller. When calculating the speed under trial conditions the propeller advance coefficieant can be found using `propeller_advance_coefficient`

**ITTC equations**: J-16

In [ ]:
#| export
def propeller_flow(
    propeller_advance_coef:float, #Propeller advance coefficient [n/a]
    rotations_sec:float, #propeller rotations per second [rev/sec]
    diameter:float, #Diamter of the propeller [m]
    )-> float: #The value that comes out is in m3/s WHAT ARE THE UNITS?
    
    "Calculate water flow into the propeller"
    
    return propeller_advance_coef * rotations_sec *diameter

really the propeller advance coefficient function is just a simple wrapper around the product of three values

In [ ]:
propeller_flow(5, 30, 2)

300

In [ ]:
#| hide

test_eq(propeller_flow(5, 30, 2),300)

In [ ]:
import nbdev; nbdev.nbdev_export()